In [ ]:
""" 
DEVELOPing stuff, 
single day anslysis
"""


In [ ]:
%load_ext autoreload
%autoreload 2

%cd ..

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.probedatTaskmodel import *
from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

# LOAD SINGLE DAY

In [ ]:
# Load filedata, quick
# a = "Pancho"
# d = 231220
# e = "gramdirstimpancho5c"
# s = 1

# mult prims
# a = "Pancho"
# d = 221125
# e = "dirshapecolor4b"
# s = 1


a = "Diego"
d = 230615
e = "priminvar5"
s = 2

fd = loadSingleDataQuick(a, d, e, s)

In [ ]:
fd

In [ ]:

for i in getIndsTrials(fd):
    print(i, "---", getTrialsUniqueTasknameGood(fd, i))

In [ ]:
fd["trials"][80]

In [ ]:
fd["params"]["n_trialoutcomes"]

# Find a filename -- a general helper, works across diff directories (e.g., server)

In [ ]:
animal = "Diego"
date = 240510
extension = "bhv2"
findFilename(animal, date, extension=extension, take_larger_file=False, return_all_files=True, return_files_sorted=True,
             return_file_sizes=True)

In [ ]:
animal = "Diego"
date = 240510
extension = "bhv2"
outdict = findFilenamesGood(animal, date, extension)
outdict

In [ ]:
filename_decide_storage_server_where_load(animal, date)

### Automatically generating dataset and making simple summaries`

In [ ]:
# Generate metadat
exptname = 'concrete2b'
date = 230426
animal = "Diego"
from analysis.dataset import generate_metadat
generate_metadat(exptname, date, date, animal, overwrite=True)    



In [ ]:
# Generate the datsaet file from raw
from analysis.dataset import generate_dataset_file_from_raw

rulelist = [str(date)]
generate_dataset_file_from_raw(animal, exptname, dataset_dir_kind="daily", rulelist=rulelist)


In [ ]:
# Plot, simple summary



##### Get trials peanut extension

In [ ]:
fd["BlockParamsByTrial"]["RunParams"].keys()

In [ ]:
# fd["TrialRecord"]["User"]["RunParams"]["BlockParams"]["10"]["params_task"]
fd["TrialRecord"]["User"]["RunParams"]["BlockParamsHotkeyUpdatedv2"]["1"]

In [ ]:
BlockParams = getTrialsBlockParamsDefault_(fd, None, "RunParams")
BlockParams["10"]["params_task"]

In [ ]:
getTrialsPeanutSampCollisExt(fd, 100)

In [ ]:
t

In [ ]:
bb

##### Plot ACTUAL peanut size (look into adapter params) [4/28/23]

In [ ]:
list_pnut = []
trials = getIndsTrials(fd)
for t in trials:
    bb = getTrialsAdapterParams(fd, t)
    if len(bb)>0:
        pnut = bb["PnutSampCollisExt"]
    else:
        pnut = np.nan
        
    list_pnut.append(pnut)
    
fig, ax = plt.subplots()
ax.plot(trials, list_pnut, "xk")

### Working with probedat

In [ ]:
from analysis.modelexpt import loadProbeDatWrapper, loadMultDataForExpt


In [ ]:
# Load
# a = "Pancho"
# d = 220328
# e = "concretechunk1"
a = "Pancho"
d = 240524
e = "primpancho1h"
# s = 1

dattoget = (e, a, d)
FD = loadMultData([dattoget])
# FD = loadMultDataForExpt(e, a, whichdates=[0,1])
PROBEDAT = loadProbeDatWrapper(FD)
P  = ProbedatTaskmodel(PROBEDAT)

P = P.clean()

##### Extract blockparams for saving dataset

In [ ]:
BlockParamsByDateSessBlock = P.extractBlockParams()

In [ ]:
BlockParamsByDateSessBlock[(220812, 1, 5)]

##### Extract rule and taskresequencibg stuff for dataset

In [ ]:
# INGORE; instead, save the blockparams

##### Conditioned on a fixed task, plot scores over day

In [ ]:
P.plotOverviewTaskPerformance

In [ ]:
Pp = P.pandas()
Pp.iloc[0]

In [ ]:
P.

In [ ]:
P.plotOverviewTaskPerformance("/tmp")

In [ ]:
P.extract_feature_list()

##### try extracting taskclass representation fo tasks (as wuld do in dataset extraction)

In [ ]:
rule = 'horiz'
extraction_params = {
    "expt":e,
    "animal":a,
    "rule":rule,
    "probedat_filter_params":{},
    "pix_add_to_sketchpad_edges":20,
#     "savedir":f"{base_dir}/database/BEH",
    "savenote":''
                    }
ProbedatFiltered = P.filterByBehPerformance(extraction_params["probedat_filter_params"])

# Reconstruct P
P = ProbedatTaskmodel(ProbedatFiltered, P.Metadat)
P = P.clean()

In [ ]:
Tasks = P.extractTasksAsClass("all")


##### Extract plan

In [ ]:
T = Tasks[-1]

T.plotTaskOnAx()
T.objects_extract()

dat = T.planclass_extract_all()


##### Convert tasks to TasksGeneral

In [ ]:
from pythonlib.drawmodel.taskgeneral import TaskClass as TaskClassGen

# convert all to general class
Tasks2 = []
for i, T in enumerate(Tasks):
    Tasks2.append(TaskClassGen())
    Tasks2[-1].initialize(ver="ml2", params=T)
Tasks = Tasks2

##### Fixing task setnum extraction

In [ ]:
from pythonlib.drawmodel.tasks import get_task_probe_info
get_task_probe_info(T.Task)

In [ ]:
T = Tasks[505]

fig, ax = plt.subplots(1,1)
T.plotTaskOnAx(ax)


##### Plot list of handpicked trials, by unique name


In [ ]:
Pp = P.pandas()

Pp["unique_task_name"]

list_names = [
    'mixture2_1-savedset-18-58507-136216', 
]
inds = Pp["unique_task_name"].isin(list_names)
inds = Pp[inds].index.tolist()

P.plotMultTrials(inds)

##### print out all tasks and N trials on those tasks


In [ ]:
# DONE! See this in preprocess code.

In [ ]:
P.print_all_tasknames("/tmp")

In [ ]:
# Filter, so only print for probe tasks
P.print_all_tasknames("/tmp", pandasfilter = {'probe':[1]})

In [ ]:
if False:
    # Doesnt do anything
    P.pandasAddBasicColumns({"unique_task_name":lambda fd, t: getTrialsUniqueTasknameGood(fd, t, nhash=8)}, if_exists="overwrite")
    Pp = P.pandas()

In [ ]:
from pythonlib.tools.pandastools import applyFunctionToAllRows
TaskClass(Pp.iloc[0]["Task"])

In [ ]:
from pythonlib.drawmodel.tasks import TaskClass
T = TaskClass([])

##### Print out scores for all trials

In [ ]:
P.print_all_feature_scores("/tmp")

In [ ]:
P.extract_feature_list_frompandas(True, False)

In [ ]:
P.print_l

In [ ]:
P.ListFeatureNames


In [ ]:
P.getTrialsHelper('getTrialsOutcomesWrapper', [0])

##### General purpose, adding new columns to Pandas

In [ ]:
# See this
P._pandasAddComputedColumns()

In [ ]:
P.ListFeatureNames


##### Overlay score thresholds on top of beh features distrubitions


In [ ]:
featurelist = P.extract_feature_list()
fdirthis = f"{fdir}/beh_eval_features"
os.makedirs(fdirthis, exist_ok=True)

blocks_to_plot = list(set(P.pandas()["block"]))
for bk in blocks_to_plot:
    filtdict = {"block":[bk]}
    PD = P.filterProbedat(filtdict, modify_in_place = False)
    if len(PD)==0:
        continue
    Pfilt = ProbedatTaskmodel(PD)
    if len(Pfilt.getIndsTrials())>3:
        featurelist_this = [f for f in featurelist if f in Pfilt.pandas().columns]
        try:
            fig = sns.pairplot(Pfilt.pandas(), vars=featurelist_this, hue="kind")
            fig.savefig(f"{fdirthis}/allfeatures_pairwise-block{bk}.pdf")
        except Exception as err:
            pass
            # print("pandas:", Pfilt.pandas())
            # print("featurelist", featurelist_this)
            # raise err
featurelist_this = [f for f in featurelist if f in P.pandas().columns]
fig = sns.pairplot(P.pandas(), vars=featurelist_this, hue="kind")
fig.savefig(f"{fdirthis}/allfeatures_pairwise.pdf")


In [ ]:
featurelist_this = [f for f in feature_list if f in P.pandas().columns]
fig = sns.pairplot(P.pandas(), vars=featurelist_this, hue="kind")


In [ ]:
P.plot_featuredists_overlying_params("/tmp")

In [ ]:
Pp

In [ ]:
col_names = {
    "ErrorCode":"getTrialsErrorCode",
    "IsAbort":"getTrialsIsAbort"
}
P.pandasAddBasicColumns(col_names)


In [ ]:
inds = P.filterPandas({"IsAbort":[False]}, return_indices=True)
P2 = P.subsampleProbedat(inds)


In [ ]:
P2.pandas()

In [ ]:
fd = P.fd(300)
T = getTrialsTask(fd, 206)

In [ ]:
getTrialsOutcomesWrapper(fd, 206)

In [ ]:
T["TaskNew"]["Objects"]["StrokindsDone"]

In [ ]:
getTrialsBlock(fd, 206)

In [ ]:
getTrialsBlock(fd, 166)

##### Plot/print peanut size.

In [ ]:
P.getTrialsHelper("getTrialsPnutSampCollisExt", [0,1])


In [ ]:
# plot over trials
cols_to_add = {"pnut_ext":P.getTrialsHelper("getTrialsPnutSampCollisExt", "all")}
Pp = P.pandas()
P.pandasAddColumns(cols_to_add)


In [ ]:
sns.scatterplot()

##### Print reason for abort (objectclass rules)

# Preprocess plots that work on fd (not on probedat)

##### Pnut size

In [ ]:
import copy

x = {1:{2:3}}
y = x
y[1][2] = 4
print(x)

x = {1:{2:3}}
y = copy.deepcopy(x)
y[1][2] = 4
print(x)

x = {1:{2:3}}
y = {k:copy.copy(v) for k, v in x.items()}
y[1][2] = 4
print(x)




In [ ]:
BlockParams = {1:{"field1":{"field2":1}}}

# BlockParams[block][field1][field2]
BlockParams2[1] = copy.deepcopy(BlockParams[1])
# BlockParams2 = BlockParams
# BlockParams2 = {k:v for k, v in BlockParams.items()}
BlockParams2[1]["field1"]["field2"] = copy.deepcopy(BlockParams[1]["field1"]["field2"])
BlockParams2[1]["field1"]["field2"] = 2

BlockParams



In [ ]:
from tools.utils import getTrialsPeanutSampCollisExt, getMultTrialsBlockParamsAllHotkeyUpdated_, getTrialsBlockParamsAllHotkeyUpdatedv2_
BP = getMultTrialsBlockParamsAllHotkeyUpdated_(fd, "RunParams")

In [ ]:
BP = getTrialsBlockParamsAllHotkeyUpdatedv2_(fd, 10, "TaskParams")
BP.keys()

In [ ]:
# Sanity check, plot peanutsize

pnut_by_block = {}
trial_by_block = {}
for trial in getIndsTrials(fd):
    
    bk = getTrialsBlock(fd, trial)
    pnutsize = getTrialsPeanutSampCollisExt(fd, trial)
    
    print(trial, bk, pnutsize)
    if bk not in pnut_by_block.keys():
        pnut_by_block[bk] = [pnutsize]
        trial_by_block[bk] = [trial]
    else:
        pnut_by_block[bk].append(pnutsize)
        trial_by_block[bk].append(trial)
        


In [ ]:
plt.figure()
list_bks = list(pnut_by_block.keys())
for bk in list_bks[10:20]:
#     bk = 39
    trials = trial_by_block[bk]
    pnuts = pnut_by_block[bk]
    plt.plot(trials, pnuts, '-o')

In [ ]:
# PRINT PNUT SIZES For each block.
{k:set(v) for k, v in pnut_by_block.items()}

In [ ]:
from pythonlib.tools.monkeylogictools import dict2list2

# x = {'1': 'transform', '2': np.array([[0.]])}
# x = {'1': {'transform':1}, '2':{"test":np.array([[0.]])}}

# x = {'1': {'transform':"test"}, '2': {'transform':"test"}}
x = {'transform':"test"}
dict2list2(x)



##### Other stuff

In [ ]:
from tools.analy import extractSessionDf

fd = P.fd(0)
df = extractSessionDf(fd)
featurestoplot = getMultTrialsBehEvalFeatures(fd)


In [ ]:
df[~(df["trial_end_method"]=="online_abort")]
plotBehSortedByScore(df, fd, "hausdorff")

##### Drawings sorted by scores, plot this separately for each block (not just all blocks)

In [ ]:
# DONE - added to preprocess.py

In [ ]:
for block in df["block"].unique():
    print(block)

    SDIR = "/tmp/trialsSortedByScore_ByBlock"
    os.makedirs(SDIR, exist_ok=True)

    blocknum = 2
    dfthis = df[df["block"]==blocknum]
    scoretypes = featurestoplot
    scoretypes.extend(["behscore", "reward"])

    for score_type in scoretypes:
        FIGS = plotBehSortedByScore(dfthis, fd, score_type)

        for ver, figs in FIGS.items():
            for i, fig in enumerate(figs):
                fig.savefig(f"{SDIR}/{score_type}_{ver}_{i}_.pdf")
                assert False

##### Convert to dataset, if desired

In [ ]:
from analysis.dataset import Probedat2Dat
pix_add_to_sketchpad_edges = 20
extraction_params = {
    "expt":e,
    "animal":a,
#                 "probedat_filter_params":{
#                     "hausdorff_filter":True,
#                     "hausdorff_filter_prctile":2.5,
#                     },
    "rule":[],
    "probedat_filter_params":{},
    "pix_add_to_sketchpad_edges":pix_add_to_sketchpad_edges,
    "savedir":"/data2/analyses/database/BEH",
    "savenote":""
}

# # ==== filter trials based on behavioral criteria, to throw out noise.
# ProbedatFiltered = P.filterByBehPerformance(extraction_params["probedat_filter_params"])

# # Reconstruct P
# P = ProbedatTaskmodel(ProbedatFiltered, P.Metadat)

# === Convert Probedat to DAT
DAT, METADAT = Probedat2Dat(P, extraction_params, save=False, keep_all_in_probedat=True)


##### To cleanup dir in preprocessing (no redundant files)

In [ ]:
a = "Pancho"
d = 240510
e = "primpancho1d"
# s = 1

In [ ]:
import glob
glob.glob(os.path.splitext(fnames[0])[0] + ".pkl")


OVERWRITE=False
filedata = loadSingleData(a, d, e, 1, resave_as_dict=False, 
                          load_resaved_data=False, resave_overwrite=False)


date, time, expt, animal, sess, ext = filename2params(fnames[0], return_ext=True)

findFilename(a, d, expt, sess, extension="pkl", return_all_files=True,
                                 doprint=False)

# f = "data2/animals/Red/210517/210517_152802_plan3_Red_1.pkl"
f = "/data2/animals/Red/210520/210520_000000_plan5_Red_1.bhv2"
pathdir, pathend = os.path.split(f)
pathname, ext = os.path.splitext(pathend)
from pythonlib.tools.expttools import fileparts, modPathFname
# fileparts(f)
modPathFname(f, "test", "test")




from tools.preprocess import cleanupDir
cleanupDir(a, d)

from pythonlib.tools.expttools import findPath
s =1
index = ["/data2/animals", [[a], [d], [d, a, s]], None, "bhv2"]
pathlist1 = findPath(*index, sort_by="date")
pathlist2 = findPath(*index, sort_by="size")

# sort by size and date. 
assert pathlist1[-1] == pathlist2[-1], "latest file must also be the largest"

for path in pathlist[:-1]:
    modPathFname(path, "IGNORE")

# Load, as in preprocess, loading h5
OVERWRITE=True
filedata = loadSingleData(a, d, e, 1, resave_as_dict=True, 
                          load_resaved_data=False, resave_overwrite=OVERWRITE)


In [ ]:
a = "Red"
d = "210605"
from tools.preprocess import cleanupDir
cleanupDir(a, d)


## RANDOM stuff

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ..

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.probedatTaskmodel import *
from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

In [ ]:
# Load filedata, quick
# a = "Pancho"
# d = 231220
# e = "gramdirstimpancho5c"
# s = 1

# mult prims
# a = "Pancho"
# d = 221125
# e = "dirshapecolor4b"
# s = 1

a = "Diego"
d = 240515
e = "primdiego1g"
s = 3

fd = loadSingleDataQuick(a, d, e, s)

In [ ]:
fd["trials"][80]

In [ ]:
fd["params"]["n_trialoutcomes"]

### Print reasons failed for objectclass failures.

In [ ]:
for t in getIndsTrialsSimple(fd):
    o = getTrialsOutcomesWrapper(fd, t)
    
    if o["trial_end_method"]=="online_abort" and o["online_abort"]["failure_mode"] == "failed_rule_objectclass":
#         print(t)
    
        T = getTrialsTaskClass(fd, t)
        T.objectclass_extract_all()
        O = T.ObjectClass
        for rule, failcount in zip(O["RuleList"], O["RuleFailureTracker"]):
            if failcount>0:
                print(t, rule[0])

In [ ]:
T.ObjectClass["RuleList"]


##### 12/21/22 - check strokes for "novel_prims"

##### ALSO: finalizing primitives extraction


In [ ]:
GLC = True
for ind in getIndsTrials(fd):
#     T = getTrialsTaskClass(fd, ind)
#     if len(T.Strokes)==0:
#         print(ind)
    print(ind)
    T = getTrialsTaskClassGeneral(fd, ind)
    
#     strokes_task = getTrialsTaskAsStrokes(fd, ind)
#     if len(strokes_task)==0:
#         print(ind)
        
#     taskstruct = getTrialsTask(fd, ind)
#     if len(taskstruct["strokes"])==0:
    T.tokens_generate(hack_is_gridlinecircle=GLC)
    
    if len(T.Strokes)>10:
        assert False

In [ ]:
T.info_extract_all_prim_versions()

In [ ]:
list_grid_ver = []
for ind in getIndsTrials(fd):
#     T = getTrialsTaskClass(fd, ind)
#     if len(T.Strokes)==0:
#         print(ind)
    T = getTrialsTaskClassGeneral(fd, ind)
    list_grid_ver.append(T.get_grid_ver())
    

In [ ]:
# Single task, print and plot useful things

ind = random.choice(getIndsTrials(fd))
plotTrialSimple(fd, ind)

T = getTrialsTaskClassGeneral(fd, ind)

display(T.get_grid_params())
display(T.get_grid_ver())
display(T.tokens_generate(hack_is_gridlinecircle=True))

##### 11/30/22 - new condensed version of blockparams hotkey updated

In [ ]:
from tools.analy import extractSessionDf

df = extractSessionDf(fd)

In [ ]:
fig, ax = plt.subplots()
# sns.lineplot(ax=ax, data=df, x="trial", y="reward_max", label="reward_max")
sns.lineplot(ax=ax, data=df, x="trial", y="reward_max")

In [ ]:
for t in getIndsTrials(fd):
    print(getTrialsBlockParamsHotkeyUpdated(fd, t)["progression"])

In [ ]:
from tools.utils import *
# RP = getTrialsBlockParamsAllHotkeyUpdated(fd, ver="RunParams")
ver = "RunParams"
trial = 500
getTrialsBlockParamsAllHotkeyUpdated(fd, trial, ver=ver, DEBUG=False)

In [ ]:
BP1 = getTrialsBlockParamsHotkeyUpdated(fd, 100)
BP2 = getTrialsBlockParamsHotkeyUpdated(fd, 100)

##### stuff

In [ ]:
getTrialsBlockParamsDefault(fd, 363, "TaskParams")["TaskParams"]

In [ ]:
getTrialsTaskParams(fd, 363).

In [ ]:
getTrialsSketchpad(fd, 363)

In [ ]:
(fd, 363)
getTrialsUniqueTasknameGood(fd, 363)

In [ ]:
for i in getIndsTrials(fd):
    print(i)
    getTrialsUniqueTasknameGood(fd, i)

##### Looking at blockparams hotkey updated

In [ ]:
fd["TrialRecord"]["User"]["RunParams"]["BlockParamsHotkeyUpdated"]["80"]["trial_of_update"]

In [ ]:
fd["TrialRecord"]["User"]["RunParams"]["BlockParamsDefaults"]["params_task"].keys()


In [ ]:
del fd["BlockParamsByTrial"]

In [ ]:
BP = getMultTrialsBlockParamsAllHotkeyUpdated_(fd, "RunParams")

In [ ]:
blocks_updated.shape

In [ ]:
vals

In [ ]:
BP[int(block)]

##### Extracting actual color used on each trial in adapters

In [ ]:
fd["TrialRecord"].keys()

In [ ]:
fd["TrialRecord"]["User"]["AdapterParams"]["5"]["guide"]["strokes"]["colors"]

In [ ]:
fd["TrialRecord"]["User"]["AdapterParams"]["5"]["bb"]["InkColorsByPt"]

##### Doing things with TrialRecord

In [ ]:
vals.squeeze()

In [ ]:
plt.figure()
vals = fd["TrialRecord"]["User"]["tasks_previously_done"]["x_character_85"]
x = np.arange(len(vals))
plt.bar(x, vals.squeeze())


In [ ]:
getTrialsBlock(fd, 500)

In [ ]:
BP = getTrialsBlockParams(fd, 500)
BP["TaskSet"]["load_old_set"]["4"]

In [ ]:
BP["probes"]

##### using TaskGeneral to load tasks and name them (matching Dataset)

In [ ]:
from tools.utils import getTrialsTaskClassGeneral, getTrialsTaskClass

for ind in getIndsTrials(fd):
    # getTrialsTaskClass(fd, ind)
    name1 = getTrialsUniqueTasknameGood(fd, ind)
    Task = getTrialsTaskClassGeneral(fd, ind)
    name2 = Task.Params["input_params"].info_generate_unique_name()
    name3 = Task.get_unique_name()
    print(ind, name1, '|', name2, '|', name3)

In [ ]:
Task.Params

##### extraction of tasks

In [ ]:
getTrialsTask(fd, 1)

In [ ]:
dict_keys(['onsets', 'guide_dot_coords', 'guide_dot_pos', 'metadat', 'area', 'str', 'stage', 'stagenum', 'alwaysnogap', 'ignore_edge_requirement', 'TaskNew', 'savedTaskSet', 'transforms', 'probe', 'feedback_ver', 'feedback_ver_prms', 'constraints_to_skip', 'ParamsMod', 'reward_multiplier', 'num_pres_even_if_no_engage', 'num_presentations', 'num_successes', 'num_fixations', 'ignore', 'strokes', 'sketchpad', 'fixpos'])

In [ ]:
for k, v in getTrialsOutcomesWrapper(fd, 9).items():
    print(k, '--',v)

In [ ]:
# Print trial outcomr for each trial
for i in getIndsTrials(fd):
    print(i, ":", getTrialsOutcomesWrapper(fd, i)["trial_end_method"])

##### Fix bug - extraction of motor timing stats.


In [ ]:
%matplotlib inline
plotTrialSingleOverview(fd, 2
                       )

### Other things

In [ ]:
for i in getIndsTrials(fd):
    print(i, getTrialsOutcomesWrapper(fd, i)["trial_end_method"])
#         print(i)

In [ ]:
plotTrialSingleOverview(fd, 434);

In [ ]:
getTrialsTouched(fd, 434)

In [ ]:
getTrialsTimesOfMotorEvents(fd, 434)

In [ ]:
getTrialsOutcomesWrapper(fd, 1)

In [ ]:
getTrialsIsAbort(fd, 145)

In [ ]:
plotTrialSingleOverview(fd, 145)

In [ ]:
getTrialsOutcomesWrapper(fd, 145)

In [ ]:
getTrialsOutcomesWrapper(fd, 434)

In [ ]:
getTrialsBehCodes(fd, 100)

In [ ]:

params_task = getTrialsBlockParamsHotkeyUpdated(fd, 100)["params_task"]["donebutton_criterion"]

In [ ]:
for i in getIndsTrials(fd):
    if getTrialsOutcomesWrapper(fd, i)["trial_end_method"]=="postscene_hotkey_abort":
        print(i, getTrialsOutcomesWrapper(fd, i)["online_abort"])

In [ ]:
for i in getIndsTrials(fd):
    if getTrialsOutcomesWrapper(fd, i)["online_abort"]["failure_mode"]=="hotkey_abort":
        print(i, getTrialsBehCodes(fd, i))


In [ ]:
for i in getIndsTrials(fd):
    print(i, getTrialsOutcomesWrapper(fd, i)["trial_end_method"])
#         print(i)

In [ ]:
getTrialsBehEvaluation(fd, 6)

In [ ]:
oc = getTrialsOutcomes

In [ ]:
fd["TrialRecord"]["User"]["AdapterParams"][f"{100}"].keys()

In [ ]:
getTrialsAdapterParams(fd, 100, 

In [ ]:
from pythonlib.tools.datetools import getDateList
a = getDateList(sdate=210320)

a[0] < 210820



In [ ]:
a = "---"
[aa for aa in a if aa=="-"]

In [ ]:
getTrialsAdapterParams(fd, i)["FailureMode"]

In [ ]:

# featurelist = getMultTrialsBehEvalFeatures(P.fd(0)) # takes first fd, but across all trials, to extract entire set of features.
# featurelist.extend(["score_final", "binary_evaluation"])

In [ ]:
P.plot_featuredists_byblock("/tmp")

In [ ]:
#### SUPERVISION PARAMS
# i = 200
# fd, t = P.fd_trial(i)
# getTrialsBlockParams(fd, t)["sequence"]


# guide = getTrialsAdapterParams(fd, t, "guide")

# guide["dynamic"].keys()
# guide["dynamic"]["version"]


# bb = getTrialsAdapterParams(fd, t)
# bb["InkColorsByPt"]

In [ ]:
a = np.asarray(1.)

isinstance(a, list)

In [ ]:
i=200
fd, t = P.fd_trial(i)
getTrialsBlockParams(fd, t)["sequence"]
bb = getTrialsAdapterParams(fd, t)

bb["abortmodes"]


####  SCRATCH, developing extraction of supervision params

In [ ]:
P.filterProbedat({"session":[2]}, True);

In [ ]:
# Place supervision features into pandas, just for easy plotting below

Pp = P.pandas()

tmp = P.getTrialsHelper("getTrialsSupervisionParams", "all")

sver = [t["sequence_ver"] for t in tmp]
Pp["sequence_ver"] = sver

Pp["col_draw"] = [t["draw_colored_strokes"] for t in tmp]
Pp["col_guide"] = [t["guide_colored_strokes"] for t in tmp]
Pp["guide_dyn"] = [t["guide_dynamic_strokes"] for t in tmp]
Pp["keep_ink_on"] = [t["keep_ink_on"] for t in tmp]
Pp["play_hit_sound"] = [t["play_hit_sound"] for t in tmp]

In [ ]:
## Examine these plots, along with looking at (video), google "behavior" spreadsheet, and blockparams in matlab
# To make sure that the extract supervisions are as expected.

sns.catplot(data=Pp, x="block", y="keep_ink_on", hue="play_hit_sound", aspect=3)
sns.catplot(data=Pp, x="block", y="sequence_ver", hue="guide_dyn", aspect=3)
sns.catplot(data=Pp, x="block", y="col_draw", hue="col_guide", aspect=3)
# sns.catplot(data=Pp, x="block", y="guide_dyn", aspect=3)

# --- useful to see if any mistakes (same block, diff params over sets?)
# sns.catplot(data=Pp, x="block", y="sequence_ver", hue="saved_setnum", aspect=3)
# sns.catplot(data=Pp, x="block", y="col_draw", hue="saved_setnum", aspect=3)
# sns.catplot(data=Pp, x="block", y="col_guide", hue="saved_setnum", aspect=3)
# sns.catplot(data=Pp, x="block", y="guide_dyn", hue="saved_setnum", aspect=3)


#### Other stuff

In [ ]:
P.clean()
P.plotOverviewBlokks("/tmp")

In [ ]:
# Try cleaning up a dir
from tools.preprocess import cleanupDir
cleanupDir("Diego", "210609")

In [ ]:
PD = P.filterProbedat({"session":[1]})


In [ ]:
P.plotOverviewBlokks("/tmp")

In [ ]:
from analysis.probedatTaskmodel import plotAllTrialsByErrorCode

sdir = "/tmp"
plotAllTrialsByErrorCode(P, sdir)


In [ ]:
df = extractSessionDf(fd)

featurestoplot = getMultTrialsBehEvalFeatures(fd)
#     featurestoplot = 
#     for key, val in getTrialsBlockParams(fd, 1)["behEval"]["beh_eval"].items():
#         if val["weight"][0][0]>0:
#             featurestoplot.append(val["feature"])
# #         if val["feature"] =="hausdorff" and val["weight"][0][0]>0:
# #             featurestoplot.append("hausdorff")
# #         if val["feature"] =="frac_touched" and val["weight"][0][0]>0:
# #             featurestoplot.append("frac_touched")
featurestoplot.append("score_offline")

fig1, fig2, fig3, fig4 = plotOverview_(df, featurestoplot=featurestoplot)


In [ ]:
plotMultTrialsSimple(fd, [200, 201], strokes_ver="peanuts")

In [ ]:
P.plo

In [ ]:
Pp.reset_index(

In [ ]:
for g in Pp.groupby(["task_stagecategory", "hausdorff_binned"]):
    print(g[0])
#     asdfd

In [ ]:
Ppsorted = Ppsorted.set_index("block", drop=True)
Ppsorted.index.names = ['index']


In [ ]:
Ppsorted

In [ ]:
Pp = P.pandas()
Ppsorted = Pp.sort_values("hausdorff")

from pythonlib.tools.pandastools import *


# fdict = {"task_stagecategory":["mixture1"]}
# out = P.filterPandas(fdict, return_indices=False)

def F(x):
    return len(x)
out, dfgroup = aggregThenReassignToNewColumn(Ppsorted, F, ["task_stagecategory"], "test", return_grouped_df=True)
out = out[out["task_stagecategory"]=="mixture2"]
set(out["test"])
# # check that indices are not changed
# print(np.all(out["trialcode"].values==P.pandas()["trialcode"].values))
# print(np.all(out.index == P.pandas().index))




# binColumn(Pp, "hausdorff", 10)


# getCount(Pp, ["task_stagecategory"], "hausdorff")

# def F(x):
#     return x["trial"]
# # applyFunctionToAllRows(Pp, F, newcolname="test")
# Ppsorted = Pp.sort_values("hausdorff").reset_index()
# applyFunctionToAllRows(Ppsorted, F, newcolname="test")


# aggregGeneral(Pp, group=["task_stagecategory", "hausdorff_binned"], 
#               values=["hausdorff_binned", "rew_total"], aggmethod=["mean"])


In [ ]:
Pp.pivot_table(index=["trial"], values=["hausdorff"])

In [ ]:
Ppdown = Pp.groupby(["task_stagecategory"]).mean().reset_index(drop=True)
Ppdown

In [ ]:
Ppsorted.reset_index().merge(Ppdown, how="left", on="task_stagecategory")

In [ ]:
# a.reset_index().merge(b, how="left").set_index('index')
Ppsorted.reset_index().merge(Ppdown, how="left", on="task_stagecategory").set_index("index")

In [ ]:
Pp.apply(

In [ ]:
def F(x):
    return x["trial"]
# applyFunctionToAllRows(Pp, F, newcolname="test")
Ppsorted = Pp.sort_values("hausdorff")
applyFunctionToAllRows(Ppsorted, F, newcolname="test")


In [ ]:
P.pandas()
tmp = pd.merge(P.pandas(), dfgroup, how="left", on=["task_stagecategory"], indicator=True)
# tmp = P.pandas().merge(dfgroup, how="left", )
tmp


In [ ]:
print(out["trialcode"].values[-15:])

print(P.pandas()["trialcode"].values[-15:])

print(tmp["trialcode"].values[-15:])

## reducing size of filedata

In [ ]:
## reducing size of fd, only keeping things needed
from pythonlib.tools.pytools import get_size
fd = P.Probedat[0]["filedata"]

In [ ]:
for k, v in fd["TrialRecord"]["User"].items():
    print("--")
    print(k)
    print(get_size(v))

In [ ]:
fd["TrialRecord"]["TaskInfo"]

In [ ]:
fd["trials"][1]["AnalogData"]["Button"]

In [ ]:
## This shows that almost all of memory of Probedat is for filedata

print("size of entire probedat")
print(get_size(P)/1000)

print("size of filedata for just the first trial")
print(get_size(P.Probedat[0]["filedata"])/1000)

print("size of filedata for just the last trial")
print(get_size(P.Probedat[-1]["filedata"])/1000)

In [ ]:
## This shows that almost all of memory of Probedat is for filedata

print("size of entire probedat")
print(get_size(P)/1000)

print("size of filedata for just the first trial")
print(get_size(P.Probedat[0]["filedata"])/1000)

print("size of filedata for just the last trial")
print(get_size(P.Probedat[-1]["filedata"])/1000)

In [ ]:
print("in filedata, size is mostly in trialrecord and trials")
for k in fd.keys():
    print(k)
    print(get_size(fd[k])/1000)
# print(get_size(fd["trials"])/1000)

In [ ]:
# trhere is only one filedata that all trials reference to.
# Confirmed taht this holds even when there are multiple days.
[id(P.Probedat[t]["filedata"]) for t in range(len(P.Probedat))]

In [ ]:
## ==== CODE TO DO REDUCING
from tools.preprocess import cleanupFiledata

print(fd["TrialRecord"].keys())
fd = P.Probedat[0]["filedata"]
cleanupFiledata(fd, True)
print(fd["TrialRecord"].keys())


##  DEVELOPING BETTER SINGLE DAY SUMMARIES

In [ ]:
SDIR = "/tmp/blokkfigs"
P.plotOverviewBlokks(SDIR)

In [ ]:
SDIR = "/data2/animals/Pancho/210311/figures/probeanalysis/tmp"
os.makedirs(SDIR, exist_ok=True)
P.plotOverviewBlokks(SDIR);
P.plotOverviewTaskPerformance(SDIR);



### compare a single task across days - check whether positions match exactlty.

In [ ]:
a = "Diego"
d = 210624
e = "linecircle4"
s = 1

In [ ]:
fd = loadSingleDataQuick(a, d, e, s)

In [ ]:
## check for a given set of tasks

FD = loadMultData([[e, a, d]]);

PD = PROBEDATfromFD(FD)

from analysis.probedatTaskmodel import *
P = ProbedatTaskmodel(PD)

In [ ]:
# Filter
if False:
    Pp = P.filterPandas({"session": [7]})
else:
    Pp = P.pandas()



In [ ]:
f = "posterior"
sns.catplot(data = Pp, x="block", y=f, aspect=2.5, kind="boxen")
sns.catplot(data = Pp, x="block", y=f, aspect=2.5, kind="point")
sns.catplot(data = Pp, x="block", y=f, aspect=2.5)

In [ ]:
P.pandas()

In [ ]:
sorted(set(P.pandas()["unique_task_name"]))

In [ ]:
f = {
    "saved_setnum":[25],
}
f = {
    "unique_task_name":["mixture2_1-savedset-25"],
}

P.filterProbedat(f)

P.pandas()

In [ ]:
plotMultTrialsSimple(P.fd(0), P.getTrialNums(), clean=True)

In [ ]:
score_type = "frac_touched"
FIGS = plotBehSortedByScore(df, fd, score_type)


In [ ]:
df = extractSessionDf(fd)
featurestoplot = getMultTrialsBehEvalFeatures(fd)
featurestoplot.append("score_offline")
featurestoplot.extend(["behscore", "reward"])
for score_type in featurestoplot:
    FIGS = plotBehSortedByScore(df, fd, score_type)


### PLOT TRIALS, LOOK AT WHY FAILED

In [ ]:
fd = P.Probedat[0]["filedata"]

In [ ]:
t = 806
P = getTrialsAdapterParams(fd, t)

getTrialsOutcomesWrapper(fd, t)


In [ ]:
P["assignstrokes"]

In [ ]:
P["DonenessTracker"]

In [ ]:
# PROBEDAT

In [ ]:
dattoget = (e, a, d)
FD = loadMultData([dattoget])


PROBEDAT = PROBEDATfromFD(FD)


from analysis.probedatTaskmodel import *

P  = ProbedatTaskmodel(PROBEDAT)

In [ ]:
blocks_to_plot = list(set(P.pandas()["block"]))
blocks_to_plot

In [ ]:
# = PAIRWISE SCATTER COMAPRING ALL BEH FEATURES
# (MOVED TO PREPROCCESS).
featurelist = getMultTrialsBehEvalFeatures(P.fd(1))
featurelist.extend(["score_final", "binary_evaluation"])

blocks_to_plot = [10]
for bk in blocks_to_plot:
    filtdict = {"block":[bk]}
    PD = P.filterProbedat(filtdict, modify_in_place = False);
    Pfilt = ProbedatTaskmodel(PD)
    fig = sns.pairplot(Pfilt.pandas(), vars=featurelist, hue="kind")
    fig.savefig("/tmp/test.pdf")



In [ ]:
sns.pairplot(P.pandas(), vars=featurelist, hue="kind")

In [ ]:
## print task names
for t in getIndsTrials(fd):
    print(getTrialsTask(fd, t)["str"])

In [ ]:
tstr = "3linePlusL_67"
for t in getIndsTrials(fd):
    task = getTrialsTask(fd, t)
    if task["str"]==tstr:
        strokes1 = getTrialsTaskAsStrokes(fd, t)

tstr = "3linePlusL_67"
for t in getIndsTrials(fd2):
    task = getTrialsTask(fd2, t)
    if task["str"]==tstr:
        strokes2 = getTrialsTaskAsStrokes(fd2, t)        

In [ ]:
# plot both overlaid
for s1, s2 in zip(strokes1, strokes2):
    assert np.all(np.isclose(s1, s2))
print("strokes match!!")

In [ ]:
print(fd["TrialRecord"]["User"]["TrialData"][f"{1}"]["CurrentTask"])

### FIXING extracting of blockparams - after splitting BlockParams to RunParams and BP


In [ ]:
# Load filedata, quick
a = "Pancho"
# d = 220218
# e = "primoverlap3c"
d = 220224
e = "chunkbyshape1"
s = 1

fd = loadSingleDataQuick(a, d, e, s)


In [ ]:
fd = loadSingleData(a, d, e, s, False, False, False)

In [ ]:
getTrialsBlockParamsDefault(fd, 100, "TaskParams")

In [ ]:
fd["params"]